# Testing endpoints

1. Fill out the Azure-specific `subscription_id`, `resource_group` and `workspace` variables in the cell below.
2. Fill out the specific `endpoint_name` and `deployment_name` you want to test

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

subscription_id = "..."
resource_group = "..."
workspace = "..."


ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Test PyTorch

In [ ]:
import torch

endpoint_name = "..."
deployment_name = "..."

t = torch.randn(1, 3, 224, 224)  # Replace with your model input shape
with open("sample-request.json", "w") as f:
    f.write(f'{{"data": {t.tolist()}}}')

ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file="sample-request.json",
)

## Test Tensorflow/ONNX

In [ ]:
import json

import numpy as np

endpoint_name = "..."
deployment_name = "..."

def numpify(t: torch.Tensor) -> np.ndarray:
    """Tensorflow and ONNX models expect NHWC format, but PyTorch uses NCHW.
    This function converts a PyTorch tensor to Numpy, and transposes the axes"""
    a = t.numpy()
    return np.transpose(a, (0, 2, 3, 1))


t = torch.randn(1, 3, 224, 224)  # Replace with your model input shape
t = numpify(t)
with open("sample-request.json", "w") as f:
    json.dump({"data": t.tolist()}, f)

ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file="sample-request.json",
)